In [1]:
import pathlib
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# params

In [9]:
data_folder = 'D:/ML_DL_AI_stuff/!DispCurves_Proj/disp_data/'

# each file in datafolder contains data 5 modes (to the moment). see 'plot_separate_disp_curv' script
# but for the first time I want tot usnly first modes
n_modes = 5 
main_folder = 'for_NN_sorted/'
train_folder = main_folder+'train/'
valid_folder = main_folder+'valid/'
test_folder = main_folder+'test/'

# functions

In [2]:
def create_img_from_csv(file):
    df_test = pd.read_csv(file, header=None)
    df_as_np = df_test.to_numpy()
    
    len_v, cols = np.shape(df_as_np)  # len of vector and number of collumns (vectors)
    # print(len_v, cols)
    
    k_test = df_as_np[:, 3][:len_v]  # wavenumber
    k_test = k_test/np.amax(k_test)
    f_test = df_as_np[:, 5][:len_v]  # frequency
    
    # plt.scatter(k_test, f_test)
    return k_test, f_test

In [3]:
def take_wave_data(file, num_curves=5, lenv=500):
    
    df = pd.read_csv(file, header=None)
    # df_as_np = df.to_numpy()
    
    freq_5_curves = []
    
    for i in range(num_curves):
        freq_5_curves.append((df[5][i:lenv+i:5]).to_numpy())
    
    return freq_5_curves

In [29]:
def create_input_curves(folder, curv_num=0):
    one_mode_ds = []
    lbls_ds = []
    
    for file in os.listdir(os.path.join(folder)):
        
        df = pd.read_csv(folder+file, header=None)
        
        # curve_data = []
        # for i in range(5):
        # curve_data.append((df[4][curv_num:500+curv_num:5]).to_numpy())
        
        one_mode_ds.append(((df[5][curv_num:500+curv_num:5])/250000).to_numpy())
        lbls_ds.append(np.array([df[0][0], df[1][0], df[2][0]]))
        
        
    return np.array(one_mode_ds), np.array(lbls_ds)

# model

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [209]:
class DiffusionModel(nn.Module):
    def __init__(self, input_dim, label_dim, hidden_dim, num_layers, num_iterations):
        super().__init__()
        self.input_dim = input_dim
        self.label_dim = label_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.num_iterations = num_iterations
        
        # adding noise layer
        self.noise_layer = nn.Linear(input_dim, input_dim)

        self.label_embedding = nn.Embedding(label_dim, hidden_dim)
        self.input_layer = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.transforms = nn.ModuleList()
        for _ in range(num_layers):
            self.transforms.append(nn.Linear(hidden_dim, hidden_dim))

    def forward(self, x, label):
        for i in range(self.num_iterations):
            # add noise to the input
            x = self.noise_layer(x) * torch.randn_like(x).float()
        
        
            # label = label.view(-1, self.label_dim)
            
            # concatenate the label embedding and the input
            label_embed = self.label_embedding(label)
            x = torch.cat([x, label_embed], dim=-1)
            x = self.input_layer(x)

            # apply the series of transformations
            for transform in self.transforms:
                x = torch.tanh(transform(x))
        return x
            
    def loss(self, x, label, observed_data):
        generated_data = self.forward(x, label)
        likelihood = torch.distributions.normal.Normal(generated_data, torch.ones_like(generated_data))
        log_likelihood = likelihood.log_prob(observed_data)
        return -torch.mean(log_likelihood)

In [210]:
nn.Embedding(label_dim, hidden_dim)

Embedding(3, 97)

In [211]:
# print(labels[10], labels[111])
# plt.plot(dataset[10])
# plt.plot(dataset[111])

In [212]:
dataset, labels = create_input_curves(data_folder+train_folder)
# convert numpy arrays to PyTorch tensors
dataset = dataset.astype(np.float32)
dataset = torch.from_numpy(dataset)# .float()
labels = torch.from_numpy(labels) # .float()

In [213]:
# create a TensorDataset
train_dataset = TensorDataset(dataset, labels)

### train parameters

In [214]:
len(dataset[0])

100

In [215]:
batch_size = 16
learning_rate = 0.0001
num_epochs = 100


input_dim = 100  # len(dataset[0])
label_dim = 3    # len(labels[0])
hidden_dim = 97  # int(input_dim*1.5)
num_layers = 3   # 5
num_iters = 300

In [216]:
# create a DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [217]:
# initialize the model
model = DiffusionModel(input_dim, label_dim, 
                       hidden_dim, num_layers, num_iters)# .double()

In [218]:
# define the loss function and optimizer
# criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [219]:
# train the model
for epoch in range(num_epochs):
    for i, (data, label) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(data, torch.tensor(label, dtype=torch.long))
        # loss = criterion(outputs, data)
        loss = model.loss(outputs, observed_data)
        loss.backward()
        optimizer.step()

C:\Users\Ksenia\AppData\Local\Temp/ipykernel_15240/2596821714.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(data, torch.tensor(label, dtype=torch.long))


RuntimeError: Tensors must have same number of dimensions: got 2 and 3

In [208]:
data.shape

torch.Size([16, 100])

In [119]:
for i, (data, label) in enumerate(train_loader):
    print(label[0][0])
    print(type(data[0][0]))
    print(type(label[0]))
    break

tensor(0.0015, dtype=torch.float64)
<class 'torch.Tensor'>
<class 'torch.Tensor'>
